### Saved Model - Format

In [2]:
#!pip install pyyaml h5py

In [3]:
import os
import sys
import yaml
import tensorflow as tf
from tensorflow import keras

2024-04-23 14:51:45.316567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from model.dataset import Dataset
from model.fp.nnfp import get_fingerprinter
from model.fp.NTxent_loss_single_gpu import NTxentLoss

In [5]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [6]:
config:str = "default" 
cfg = load_config(config)

cli: Configuration from ./config/default.yaml


In [7]:
from model.fp.nnfp import test
import numpy as np
from model.fp.nnfp import get_fingerprinter

In [8]:
test()

In [9]:
# -*- coding: utf-8 -*-
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
""" nnfp.py

'Neural Audio Fingerprint for High-specific Audio Retrieval based on 
Contrastive Learning', https://arxiv.org/abs/2010.11910

USAGE:
    
    Please see test() in the below.
    
"""
import numpy as np
import tensorflow as tf
assert tf.__version__ >= "2.0"


class ConvLayer(tf.keras.layers.Layer):
    """
    Separable convolution layer
    
    Arguments
    ---------
    hidden_ch: (int)
    strides: [(int, int), (int, int)]
    norm: 'layer_norm1d' for normalization on Freq axis. (default)
          'layer_norm2d' for normalization on on FxT space 
          'batch_norm' or else, batch-normalization
    
    Input
    -----
    x: (B,F,T,1)
    
    [Conv1x3]>>[ELU]>>[BN]>>[Conv3x1]>>[ELU]>>[BN]
    
    Output
    ------
    x: (B,F,T,C) with {F=F/stride, T=T/stride, C=hidden_ch}
    
    """
    def __init__(self,
                 hidden_ch=128,
                 strides=[(1,1),(1,1)],
                 norm='layer_norm2d'):
        super(ConvLayer, self).__init__()
        self.conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                                 kernel_size=(1, 3),
                                                 strides=strides[0],
                                                 padding='SAME',
                                                 dilation_rate=(1, 1),
                                                 kernel_initializer='glorot_uniform',
                                                 bias_initializer='zeros')
        self.conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                                 kernel_size=(3, 1),
                                                 strides=strides[1],
                                                 padding='SAME',
                                                 dilation_rate=(1, 1),
                                                 kernel_initializer='glorot_uniform',
                                                 bias_initializer='zeros')
        
        if norm == 'layer_norm1d':
            self.BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
            self.BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
        elif norm == 'layer_norm2d':
            self.BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
            self.BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        else:
            self.BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1) # Fix axis: 2020 Apr20
            self.BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
            
        self.forward = tf.keras.Sequential([self.conv2d_1x3,
                                            tf.keras.layers.ELU(),
                                            self.BN_1x3,
                                            self.conv2d_3x1,
                                            tf.keras.layers.ELU(),
                                            self.BN_3x1
                                            ])

       
    def call(self, x):
        return self.forward(x)


class DivEncLayer(tf.keras.layers.Layer):
    """
    Multi-head projection a.k.a. 'divide and encode' layer:
        
    • The concept of 'divide and encode' was discovered  in Lai et.al.,
     'Simultaneous Feature Learning and Hash Coding with Deep Neural Networks',
      2015. https://arxiv.org/abs/1504.03410
    • It was also adopted in Gfeller et.al. 'Now Playing: Continuo-
      us low-power music recognition', 2017. https://arxiv.org/abs/1711.10958
    
    Arguments
    ---------
    q: (int) number of slices as 'slice_length = input_dim / q'
    unit_dim: [(int), (int)]
    norm: 'layer_norm1d' or 'layer_norm2d' uses 1D-layer normalization on the feature.
          'batch_norm' or else uses batch normalization. Default is 'layer_norm2d'.

    Input
    -----
    x: (B,1,1,C)
    
    Returns
    -------
    emb: (B,Q)
    
    """
    def __init__(self, q=128, unit_dim=[32, 1], norm='batch_norm'):
        super(DivEncLayer, self).__init__()

        self.q = q
        self.unit_dim = unit_dim
        self.norm = norm
        
        if norm in ['layer_norm1d', 'layer_norm2d']:
            self.BN = [tf.keras.layers.LayerNormalization(axis=-1) for i in range(q)]
        else:
            self.BN = [tf.keras.layers.BatchNormalization(axis=-1) for i in range(q)]
            
        self.split_fc_layers = self._construct_layers() 


    def build(self, input_shape):
        # Prepare output embedding variable for dynamic batch-size 
        self.slice_length = int(input_shape[-1] / self.q)

 
    def _construct_layers(self):
        layers = list()
        for i in range(self.q): # q: num_slices
            layers.append(tf.keras.Sequential([tf.keras.layers.Dense(self.unit_dim[0], activation='elu'),
                                               #self.BN[i],
                                               tf.keras.layers.Dense(self.unit_dim[1])]))
        return layers

 
    @tf.function
    def _split_encoding(self, x_slices):
        """
        Input: (B,Q,S)
        Returns: (B,Q)
        
        """
        out = list()
        for i in range(self.q):
            out.append(self.split_fc_layers[i](x_slices[:, i, :]))
        return tf.concat(out, axis=1)

    
    def call(self, x): # x: (B,1,1,2048)
        x = tf.reshape(x, shape=[x.shape[0], self.q, -1]) # (B,Q,S); Q=num_slices; S=slice length; (B,128,8 or 16)
        return self._split_encoding(x)
    
    
class FingerPrinter(tf.keras.Model):
    """
    Fingerprinter: 'Neural Audio Fingerprint for High-specific Audio Retrieval
        based on Contrastive Learning', https://arxiv.org/abs/2010.11910
    
    IN >> [Convlayer]x8 >> [DivEncLayer] >> [L2Normalizer] >> OUT 
    
    Arguments
    ---------
    input_shape: tuple (int), not including the batch size
    front_hidden_ch: (list)
    front_strides: (list)
    emb_sz: (int) default=128
    fc_unit_dim: (list) default=[32,1]
    norm: 'layer_norm1d' for normalization on Freq axis. 
          'layer_norm2d' for normalization on on FxT space (default).
          'batch_norm' or else, batch-normalization.
    use_L2layer: True (default)
    
    • Note: batch-normalization will not work properly with TPUs.
                    
    
    Input
    -----
    x: (B,F,T,1)
    
        
    Returns
    -------
    emb: (B,Q) 
    
    """
    def __init__(self,
                 input_shape=(256,32,1),
                 front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                 front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                [(1,2), (2,1)], [(1,2), (2,1)],
                                [(1,1), (2,1)], [(1,2), (2,1)],
                                [(1,1), (2,1)], [(1,2), (2,1)]],
                 emb_sz=128, # q
                 fc_unit_dim=[32,1],
                 norm='layer_norm2d',
                 use_L2layer=True):
        super(FingerPrinter, self).__init__()
        self.front_hidden_ch = front_hidden_ch
        self.front_strides = front_strides
        self.emb_sz=emb_sz
        self.norm = norm
        self.use_L2layer = use_L2layer
        
        self.n_clayers = len(front_strides)
        self.front_conv = tf.keras.Sequential(name='ConvLayers')
        if ((front_hidden_ch[-1] % emb_sz) != 0):
            front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz                
        
        # Front (sep-)conv layers
        #x = tf.zeros((1,)+ input_shape, dtype=tf.float32)
        #print(f"ConvLayer entrada: {self.front_conv(x).shape}")
        for i in range(self.n_clayers):
            self.front_conv.add(ConvLayer(hidden_ch=front_hidden_ch[i],
                strides=front_strides[i], norm=norm))
            #print(f"ConvLayer {i+1}: {self.front_conv(x).shape}")
        self.front_conv.add(tf.keras.layers.Flatten()) # (B,F',T',C) >> (B,D)
            
        # Divide & Encoder layer
        self.div_enc = DivEncLayer(q=emb_sz, unit_dim=fc_unit_dim, norm=norm)

        
    @tf.function
    def call(self, inputs):
        x = self.front_conv(inputs) # (B,D) with D = (T/2^4) x last_hidden_ch
        x = self.div_enc(x) # (B,Q)
        if self.use_L2layer:
            return tf.math.l2_normalize(x, axis=1) 
        else:
            return x


In [10]:
def get_fingerprinter(cfg, trainable=False):
    """
    Input length : 1s or 2s
    
    Arguements
    ----------
    cfg : (dict)
        created from the '.yaml' located in /config dicrectory

    Returns
    -------
    <tf.keras.Model> FingerPrinter object
    
    """
    input_shape = (256, 32, 1) 
    emb_sz = cfg['MODEL']['EMB_SZ']
    norm = cfg['MODEL']['BN']
    fc_unit_dim = [32, 1]
    
    m = FingerPrinter(input_shape=input_shape,
                      emb_sz=emb_sz,
                      fc_unit_dim=fc_unit_dim,
                      norm=norm)
    m.trainable = trainable
    return m

In [11]:
input_1s = tf.constant(np.random.randn(3,256,32,1), dtype=tf.float32) # BxFxTx1

In [12]:
fprinter = FingerPrinter(emb_sz=128, fc_unit_dim=[32, 1], norm='layer_norm2d')

In [13]:
emb_1s = fprinter(input_1s) # BxD

In [14]:
fprinter.summary()

Model: "finger_printer_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ConvLayers (Sequential)     (3, 1024)                 16897920  
                                                                 
 div_enc_layer_2 (DivEncLay  multiple                  41088     
 er)                                                             
                                                                 
Total params: 16939008 (64.62 MB)
Trainable params: 16939008 (64.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model = fprinter

# Carregar os arquivos de checkpoint usando tf.train.Checkpoint
checkpoint = tf.train.Checkpoint(model=model)

checkpoint.restore('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/ckpt-100.index')

model = checkpoint.model

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


In [16]:
model.summary()

Model: "finger_printer_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ConvLayers (Sequential)     (3, 1024)                 16897920  
                                                                 
 div_enc_layer_2 (DivEncLay  multiple                  41088     
 er)                                                             
                                                                 
Total params: 16939008 (64.62 MB)
Trainable params: 16939008 (64.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

TypeError: in user code:

    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/saving/legacy/saving_utils.py", line 147, in _wrapped_model  *
        outputs = model(*args, **kwargs)
    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_fileg7is_tgz.py", line 11, in tf__call
        x = ag__.converted_call(ag__.ld(self).div_enc, (ag__.ld(x),), None, fscope)
    File "/tmp/__autograph_generated_filedqkzi9ia.py", line 10, in tf__call
        x = ag__.converted_call(ag__.ld(tf).reshape, (ag__.ld(x),), dict(shape=[ag__.ld(x).shape[0], ag__.ld(self).q, -1]), fscope)

    TypeError: Exception encountered when calling layer 'finger_printer_2' (type FingerPrinter).
    
    in user code:
    
        File "/home/rodrigo/Documents/neural-audio-fp/model/fp/nnfp.py", line 230, in call  *
         

In [16]:
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

TypeError: in user code:

    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/saving/legacy/saving_utils.py", line 147, in _wrapped_model  *
        outputs = model(*args, **kwargs)
    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file7lemko3n.py", line 11, in tf__call
        x = ag__.converted_call(ag__.ld(self).div_enc, (ag__.ld(x),), None, fscope)
    File "/tmp/__autograph_generated_file91qw6ou_.py", line 10, in tf__call
        x = ag__.converted_call(ag__.ld(tf).reshape, (ag__.ld(x),), dict(shape=[ag__.ld(x).shape[0], ag__.ld(self).q, -1]), fscope)

    TypeError: Exception encountered when calling layer 'finger_printer_2' (type FingerPrinter).
    
    in user code:
    
        File "/tmp/ipykernel_944702/1772340225.py", line 230, in call  *
            x = self.div_enc(x

In [ ]:
tf.keras.models.save_model(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

In [23]:
#checkpoint.restore('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/ckpt-100.index')

In [22]:
# Carregar os arquivos de checkpoint usando tf.train.Checkpoint
checkpoint = tf.train.Checkpoint(model=fprinter)
#checkpoint.restore(tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'))
checkpoint.restore(tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/ckpt-100.index'))
checkpoint

# Obtendo o modelo do checkpoint
model = checkpoint.model

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

TypeError: in user code:

    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/saving/legacy/saving_utils.py", line 147, in _wrapped_model  *
        outputs = model(*args, **kwargs)
    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filerj_1icve.py", line 11, in tf__call
        x = ag__.converted_call(ag__.ld(self).div_enc, (ag__.ld(x),), None, fscope)
    File "/tmp/__autograph_generated_filed_0ec1gm.py", line 10, in tf__call
        x = ag__.converted_call(ag__.ld(tf).reshape, (ag__.ld(x),), dict(shape=[ag__.ld(x).shape[0], ag__.ld(self).q, -1]), fscope)

    TypeError: Exception encountered when calling layer 'finger_printer' (type FingerPrinter).
    
    in user code:
    
        File "/tmp/ipykernel_933778/1772340225.py", line 230, in call  *
            x = self.div_enc(x) 

In [24]:
def load_checkpoint(checkpoint_root_dir, checkpoint_name, checkpoint_index,
                    m_fp):
    """ Load a trained fingerprinter """
    # Create checkpoint
    checkpoint = tf.train.Checkpoint(model=m_fp)
    checkpoint_dir = checkpoint_root_dir + f'/{checkpoint_name}/'
    c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir,
                                           max_to_keep=None)

    # Load
    if checkpoint_index == None:
        tf.print("\x1b[1;32mArgument 'checkpoint_index' was not specified.\x1b[0m")
        tf.print('\x1b[1;32mSearching for the latest checkpoint...\x1b[0m')
        latest_checkpoint = c_manager.latest_checkpoint
        if latest_checkpoint:
            checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
            status = checkpoint.restore(latest_checkpoint)
            status.expect_partial()
            tf.print(f'---Restored from {c_manager.latest_checkpoint}---')
        else:
            raise FileNotFoundError(f'Cannot find checkpoint in {checkpoint_dir}')
    else:
        checkpoint_fpath = checkpoint_dir + 'ckpt-' + str(checkpoint_index)
        status = checkpoint.restore(checkpoint_fpath) # Let TF to handle error cases.
        status.expect_partial()
        tf.print(f'---Restored from {checkpoint_fpath}---')
    return checkpoint_index

In [25]:
checkpoint_name:str = 'CHECKPOINT_BSZ_120'
checkpoint_index:int = 100

In [26]:
checkpoint_root_dir = cfg['DIR']['LOG_ROOT_DIR'] + 'checkpoint/'
checkpoint_index = load_checkpoint(checkpoint_root_dir, checkpoint_name, checkpoint_index, model)

---Restored from ./logs/checkpoint//CHECKPOINT_BSZ_120/ckpt-100---


old

In [8]:
from model.trainer import build_fp

In [9]:
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer

In [10]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    #return m_pre, m_specaug, m_fp
    return m_pre, m_specaug, m_fp

In [11]:
m_pre, m_specaug, m_fp = build_fp(cfg)

In [12]:
dataset = Dataset(cfg)

In [43]:
lr_schedule = tf.keras.experimental.CosineDecay(
            initial_learning_rate=float(cfg['TRAIN']['LR']),
            decay_steps=1,
            alpha=1e-06)

In [44]:
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [45]:
m_fp.compile(optimizer=opt,
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[tf.metrics.SparseCategoricalAccuracy()])

In [ ]:
m_fp.summary()

In [ ]:
tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Defina a estrutura do seu modelo
# Exemplo:
# model = create_model()
model=m_fp
input_shape = (256,32,1)

model.build((None,) + input_shape)

# Carregar os arquivos de checkpoint usando tf.train.Checkpoint
checkpoint = tf.train.Checkpoint(model=model)
#checkpoint.restore(tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'))
checkpoint.restore('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/ckpt-100.index')

# Obtendo o modelo do checkpoint
model = checkpoint.model

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

In [ ]:
# Carregar o último ponto de verificação (checkpoint)
latest_checkpoint = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')
model = load_model(latest_checkpoint)

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

In [19]:
checkpoint_dir = '/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'
os.listdir(checkpoint_dir)

['ckpt-35.data-00000-of-00001',
 'checkpoint',
 'ckpt-67.data-00000-of-00001',
 'ckpt-35.index',
 'ckpt-59.index',
 'ckpt-55.data-00000-of-00001',
 'ckpt-95.data-00000-of-00001',
 'ckpt-75.data-00000-of-00001',
 'ckpt-100.index',
 'ckpt-3.index',
 'ckpt-91.index',
 'ckpt-79.index',
 'ckpt-59.data-00000-of-00001',
 'ckpt-99.data-00000-of-00001',
 'ckpt-99.index',
 'ckpt-3.data-00000-of-00001',
 'ckpt-87.data-00000-of-00001',
 'ckpt-39.index',
 'ckpt-27.data-00000-of-00001',
 'ckpt-15.index',
 'ckpt-27.index',
 'ckpt-71.index',
 'ckpt-15.data-00000-of-00001',
 'ckpt-19.data-00000-of-00001',
 'ckpt-87.index',
 'ckpt-51.index',
 'ckpt-91.data-00000-of-00001',
 'ckpt-51.data-00000-of-00001',
 'ckpt-98.index',
 'ckpt-100.data-00000-of-00001',
 'ckpt-95.index',
 'ckpt-98.data-00000-of-00001',
 'ckpt-83.data-00000-of-00001',
 'ckpt-75.index',
 'ckpt-47.index',
 'ckpt-79.data-00000-of-00001',
 'ckpt-83.index',
 'ckpt-67.index',
 'ckpt-43.data-00000-of-00001',
 'ckpt-43.index',
 'ckpt-23.index',

In [19]:
latest = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [8]:
model = get_fingerprinter(cfg, trainable=True)

In [47]:
dataset = Dataset(cfg)
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])

In [51]:
model.compile(optimizer='ADAM', loss=loss_obj_train, metrics=['accuracy', 'loss_weights='])

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model.load_weights(latest)